## Study Cluster Synchronization in the integration of the FHN model.

### First, a test

Based on what is shown in the paper (https://doi.org/10.48550/arXiv.2303.08668), first we test with a complete graph.

In [1]:
include("src/msf.jl")
zero_msf = msf_zero()
println("zero_msf = ", zero_msf)
println("msf = ", master_stability_function(zero_msf, 0))

zero_msf = 0.20373021436669875
msf = 

6.6612382763443796e-18


In [2]:
include("src/network_matrices_creation.jl")
include("src/cluster_synch.jl")
include("src/network_simulation.jl")

N = 10
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
G = test_matrix_for_cluster_synch();
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
eigenvalues = unique(round.(eigenvalues, digits=8))[2:end]
critical_couplings = zero_msf./eigenvalues #unique(zero_msf./eigenvalues)[2:end]#
println("Critical Couplings: ", critical_couplings)
println("Eigenvalues: ", eigenvalues)
σ = critical_couplings[3]*1.1


Critical Couplings: [0.020373021436669875, 0.011984130256864633, 0.009701438779366607]
Eigenvalues: [10.0, 17.0, 21.0]


0.01067158265730327

In [53]:
include("src/network_simulation.jl")
using Random
x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])
sol = solve(prob; dtmax=0.9);

In [54]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

colors = ["#5ec962", "#21918c", "#3b528b"]
f = Figure(size= (1000, 600))
uni_clusters = unique(clusters)
for (i, cluster) in enumerate(uni_clusters)
    if isempty(cluster)
        continue
    else
        if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
            cluster = setdiff(cluster[1], uni_clusters[i+1][1])
        else
            cluster = cluster[1]
        end
        println("Cluster $i: $cluster")

        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $cluster", linewidth=1, color = colors[i])
        axislegend()
    end
end
display(f);

In [55]:
st_plot(sol, 500, 1000;skip_every=1)

In [23]:
using GLMakie
using Trapz
using ProgressMeter
using Base.Threads

include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

N_d = 50
N_realizations = 100
d_sweep = range(0.005, 0.025, length=N_d)
t_measurement = 300.0
t_transient = 0.0
cluster_synch_averages = zeros(N_d, 3)
global_synch_averages = zeros(N_d)

f = Figure(size=(800, 600))
uni_clusters = unique(clusters)
@showprogress for k in 1:N_d
    σ = d_sweep[k]
    temp = zeros(N_realizations, 3)
    for realization in 1:N_realizations
        x_0 = zeros(2*N) + 0.0001 .* randn(2*N)
        prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, t_transient + t_measurement), [a, eps, σ])
        # alg = Tsit5()
        sol = solve(prob; dtmax=0.5)
        t_val, global_synch = synch_error_time_series(sol)
        global_synch_averages[k] += trapz(t_val[findfirst(t_val .> t_transient):end], global_synch[findfirst(t_val .> t_transient):end])/(t_measurement * N_realizations)
        for (i, cluster) in enumerate(uni_clusters)
            if isempty(cluster)
                continue
            else
                if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
                    cluster = setdiff(cluster[1], uni_clusters[i+1][1])
                else
                    cluster = cluster[1]
                end
                t_values, synch_error = local_synch_error(sol, cluster)
                # cluster_synch_averages[k, i] += trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
                temp[realization, i] = trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
            end
        end
    end
    cluster_synch_averages[k, :] = sum(temp, dims=1)
end

ax = Axis(f[1, 1])
ax.xlabel = "Coupling Strength"
ax.ylabel = "Average Synch Error"
for i in 1:3
    cluster = uni_clusters[i]
    if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
        cluster = setdiff(cluster[1], uni_clusters[i+1][1])
    else
        cluster = cluster[1]
    end
    lines!(ax, d_sweep, cluster_synch_averages[:, i]; label="Cluster $cluster", linewidth=1)
end
lines!(ax, d_sweep, global_synch_averages; label="Global", linestyle=:dash)
vlines!(ax, critical_couplings; label="Critical Couplings", linewidth=1, color = :red)
axislegend()
display(f);


Progress:   4%|█▋                                       |  ETA: 0:06:41

Progress:   6%|██▌                                      |  ETA: 0:06:33

Progress:   8%|███▎                                     |  ETA: 0:06:25

Progress:  10%|████▏                                    |  ETA: 0:06:17

Progress:  12%|████▉                                    |  ETA: 0:06:07

...

Progress:  92%|█████████████████████████████████████▊   |  ETA: 0:00:29

Progress:  94%|██████████████████████████████████████▌  |  ETA: 0:00:22

Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:00:15

Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:07

Progress: 100%|█████████████████████████████████████████| Time: 0:06:02


In [9]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 200))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

### Let's see if we can spot hysteresis